In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os
import sys
!{sys.executable} -m pip install pyarrow

import pandas as pd

df = pd.read_parquet(r"C:\KDT\bok_pjt\team2_forecast_pjt\db\preprocessing\preprocessed_260106.parquet")

In [3]:
# tokens 리스트화
import ast
def convert_to_list(x):
    if isinstance(x, str): 
        return ast.literal_eval(x) 
    return x

df['tokens'] = df['tokens'].apply(convert_to_list)
print(f"token data 타입: {type(df['tokens'].iloc[0])}")


token data 타입: <class 'list'>


In [4]:
# 콜금리 데이터 정제
rate_df = pd.read_csv(r'C:\KDT\bok_pjt\team2_forecast_pjt\db\rates\call_rate.csv')
rate_df['date'] = pd.to_datetime(rate_df['date'].str.replace('.', '-'))
rate_df = rate_df.sort_values('date')
rate_df

,date,call_rate
0,2011-12-30,3.290
1,2012-01-02,3.250
2,2012-01-03,3.250
3,2012-01-04,3.240
4,2012-01-05,3.250
...,...,...
3449,2025-12-23,2.547
3450,2025-12-24,2.543
3451,2025-12-26,2.553
3452,2025-12-29,2.522


In [5]:
df_study = df.copy()
df_study = df_study.sort_values('date')
df_study['date'] = pd.to_datetime(df_study['date'])
df_study['date_1m'] = df_study['date'] + timedelta(days=30)
df_study

,doc_id,sent_id,date,content,tokens,category,source,date_1m
4619427,237430,4,2012-01-01,하지만 시장 전문가들은 내년 중순 이후 금값이 올해 최고치인 온스당 1920달러를 ...,[하/VV;있/VV;보/VV],뉴스,한국경제,2012-01-31
4616880,237283,17,2012-01-01,국가채무위기란 국가가 빌린 돈을 제때 갚지 못해 기업처럼 부도가 날 수 있는 상황을...,[날/VV;있/VV],뉴스,한국경제,2012-01-31
4616879,237283,50,2012-01-01,국가채무위기의 해결방법은 없을까 방법은 있지만 쉽지 않아요.,[],뉴스,한국경제,2012-01-31
4616878,237283,35,2012-01-01,그리스가 유로에 가입하지 않았다면 그랬다면 그리스와 독일 간의 환율이 변동하면서 경...,[있/VV],뉴스,한국경제,2012-01-31
4616877,237283,6,2012-01-01,A. 이탈리아로 유학간 여자친구 걱정은 크게 안해도 될 것 같습니다.,[],뉴스,한국경제,2012-01-31
...,...,...,...,...,...,...,...,...
2880488,158861,55,2025-12-30,또 올해 트리컬러와 퍼스트브랜드의 파산으로 일부 금융 회사가 대출 채권을 날린 사례...,[들/VV],뉴스,한국경제,2026-01-29
2880487,158861,54,2025-12-30,이밖에 블룸버그는 지난 해 엄청난 수익을 냈던 터키 리라화 캐리 트레이드가 올해 정...,[],뉴스,한국경제,2026-01-29
2880486,158861,24,2025-12-30,K랠리 한국 주식의 급등 올해 한국 증시는 반전과 스릴 넘치는 전개로 전세계 주요 ...,[],뉴스,한국경제,2026-01-29
2880464,158861,1,2025-12-30,전반적 상승세속 레버리지 투자 많아 수익과 급변동 많아 사진 REUTERS 블룸버그...,[],뉴스,한국경제,2026-01-29


In [6]:
df_study = pd.merge_asof(df_study, rate_df, on='date', direction='backward')
df_study = df_study.rename(columns={'call_rate': 'rate_today'})

df_study = pd.merge_asof(df_study, rate_df, left_on='date_1m', right_on='date', direction='forward')
df_study = df_study.rename(columns={'call_rate': 'rate_1m'}).drop(columns='date_y', errors='ignore')
df_study = df_study.rename(columns={'date_x': 'date'})

df_study

,doc_id,sent_id,date,content,tokens,category,source,date_1m,rate_today,rate_1m
0,237430,4,2012-01-01,하지만 시장 전문가들은 내년 중순 이후 금값이 올해 최고치인 온스당 1920달러를 ...,[하/VV;있/VV;보/VV],뉴스,한국경제,2012-01-31,3.29,3.27
1,237283,17,2012-01-01,국가채무위기란 국가가 빌린 돈을 제때 갚지 못해 기업처럼 부도가 날 수 있는 상황을...,[날/VV;있/VV],뉴스,한국경제,2012-01-31,3.29,3.27
2,237283,50,2012-01-01,국가채무위기의 해결방법은 없을까 방법은 있지만 쉽지 않아요.,[],뉴스,한국경제,2012-01-31,3.29,3.27
3,237283,35,2012-01-01,그리스가 유로에 가입하지 않았다면 그랬다면 그리스와 독일 간의 환율이 변동하면서 경...,[있/VV],뉴스,한국경제,2012-01-31,3.29,3.27
4,237283,6,2012-01-01,A. 이탈리아로 유학간 여자친구 걱정은 크게 안해도 될 것 같습니다.,[],뉴스,한국경제,2012-01-31,3.29,3.27
...,...,...,...,...,...,...,...,...,...,...
4974762,158861,55,2025-12-30,또 올해 트리컬러와 퍼스트브랜드의 파산으로 일부 금융 회사가 대출 채권을 날린 사례...,[들/VV],뉴스,한국경제,2026-01-29,2.54,NaN
4974763,158861,54,2025-12-30,이밖에 블룸버그는 지난 해 엄청난 수익을 냈던 터키 리라화 캐리 트레이드가 올해 정...,[],뉴스,한국경제,2026-01-29,2.54,NaN
4974764,158861,24,2025-12-30,K랠리 한국 주식의 급등 올해 한국 증시는 반전과 스릴 넘치는 전개로 전세계 주요 ...,[],뉴스,한국경제,2026-01-29,2.54,NaN
4974765,158861,1,2025-12-30,전반적 상승세속 레버리지 투자 많아 수익과 급변동 많아 사진 REUTERS 블룸버그...,[],뉴스,한국경제,2026-01-29,2.54,NaN


In [7]:
df_study['diff'] = df_study['rate_1m'] - df_study['rate_today']
df_study['label'] = df_study['diff'].apply(lambda x: 'hawkish' if x > 0.03 else ('dovish' if x < -0.03 else 'neutral'))

df_study = df_study[df_study['tokens'].map(len) > 0]
df_study

,doc_id,sent_id,date,content,tokens,category,source,date_1m,rate_today,rate_1m,diff,label
0,237430,4,2012-01-01,하지만 시장 전문가들은 내년 중순 이후 금값이 올해 최고치인 온스당 1920달러를 ...,[하/VV;있/VV;보/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
1,237283,17,2012-01-01,국가채무위기란 국가가 빌린 돈을 제때 갚지 못해 기업처럼 부도가 날 수 있는 상황을...,[날/VV;있/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
3,237283,35,2012-01-01,그리스가 유로에 가입하지 않았다면 그랬다면 그리스와 독일 간의 환율이 변동하면서 경...,[있/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
5,237283,7,2012-01-01,3년 전에 비해 유로화 가치가 원화 대비 많이 떨어졌거든요.,[많이/MAG;떨어/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
20,237283,28,2012-01-01,여행객도 크게 늘고 덩달아 각국의 여행 수입도 늘어났죠.,[늘/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
4974742,158861,38,2025-12-30,막대한 국채를 보유한 일본이 곧 금리를 올릴 수 밖에 없을 것으로 보고 공매도에 들...,[많/VA],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral
4974744,158861,22,2025-12-30,그 달에 비트코인과 금은 모두 사상 최고치를 경신했다.,[최고/NNG;경신/NNG;하/VV],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral
4974749,1621,15,2025-12-30,AI 버블이 곧 터질지에 대한 질문에는 버블이 있지만 관리 가능한 수준 이라는 응답...,[가장/MAG;많/VA],뉴스,매일경제,2026-01-29,2.54,NaN,NaN,neutral
4974758,158861,13,2025-12-30,2008년 서브프라임 모기지 거품을 예언해 유명해진 마이클 버리가 운영하는 사이언 ...,[공시/NNG;하/VV],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral


In [8]:
# 데이터 중간 점검
check_nan = df_study[['rate_today', 'rate_1m', 'diff', 'label']].isnull().sum()

print("--- 빈칸(NaN/NaT) 개수 확인 ---")
print(check_nan)

--- 빈칸(NaN/NaT) 개수 확인 ---
rate_today        0
rate_1m       12597
diff          12597
label             0
dtype: int64


In [9]:
# 점검
label_counts = df_study['label'].value_counts()
label_ratio = df_study['label'].value_counts(normalize=True) * 100

print("\n--- 라벨별 개수 및 비중 ---")
for label in label_counts.index:
    print(f"[{label}]: {label_counts[label]}개 ({label_ratio[label]:.2f}%)")


--- 라벨별 개수 및 비중 ---
[neutral]: 1149896개 (54.75%)
[hawkish]: 485046개 (23.09%)
[dovish]: 465484개 (22.16%)


In [15]:
# 추후 tone 점수 산출용 df 저장
result_path = r"C:\KDT\bok_pjt\team2_forecast_pjt\db\tone"
if not os.path.exists(result_path):
    os.makedirs(result_path)

file_name = "df_for_tone_260107.parquet"
full_path = os.path.join(result_path, file_name)

df_study.to_parquet(full_path, index=False, engine='pyarrow')

In [10]:
# 사전 학습용 df 만들기
df_lex = df_study.copy()
df_lex = df_lex.dropna(subset=['rate_1m'])
df_lex = df_lex[df_lex['tokens'].map(len) > 0]
df_lex = df_lex[df_lex['label'] != 'neutral']

check_nan = df_lex[['rate_today', 'rate_1m', 'diff', 'label']].isnull().sum()
print(check_nan)

rate_today    0
rate_1m       0
diff          0
label         0
dtype: int64


In [11]:
label_counts = df_lex['label'].value_counts()
label_ratio = df_lex['label'].value_counts(normalize=True) * 100

print("\n--- 라벨별 개수 및 비중 ---")
for label in label_counts.index:
    print(f"[{label}]: {label_counts[label]}개 ({label_ratio[label]:.2f}%)")


--- 라벨별 개수 및 비중 ---
[hawkish]: 485046개 (51.03%)
[dovish]: 465484개 (48.97%)


### NBC 모델 학습

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np

df_test = df_lex

n_test_rounds = 30
bagging_results = []

for i in range(n_test_rounds):
    train_data = df_test.sample(frac=0.9, random_state=i)
    cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False, preprocessor=lambda x: x)
    X_train = cv.fit_transform(train_data['tokens'])
    y_train = train_data['label']

    nbc = MultinomialNB()
    nbc.fit(X_train, y_train)

    dovish_idx = np.where(nbc.classes_ == 'dovish')[0][0]
    hawkish_idx = np.where(nbc.classes_ == 'hawkish')[0][0]

    prob_hawkish = np.exp(nbc.feature_log_prob_[hawkish_idx])
    prob_dovish = np.exp(nbc.feature_log_prob_[dovish_idx])

    polarity_ratio = prob_hawkish / (prob_dovish + 1e-10)
    words = cv.get_feature_names_out()
    score_df = pd.DataFrame({'word': words, f'score_{i}': polarity_ratio}).set_index('word')
    bagging_results.append(score_df)
    
    print(f"{i+1}번째 학습 완료")

final_lexicon = pd.concat(bagging_results, axis=1)
final_lexicon['polarity_score'] = final_lexicon.mean(axis=1)

c:\Users\hg432\anaconda3\envs\nlp_study\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


1번째 학습 완료


c:\Users\hg432\anaconda3\envs\nlp_study\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


2번째 학습 완료


c:\Users\hg432\anaconda3\envs\nlp_study\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

28번째 학습 완료


c:\Users\hg432\anaconda3\envs\nlp_study\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


29번째 학습 완료


c:\Users\hg432\anaconda3\envs\nlp_study\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


30번째 학습 완료


In [13]:
# intensity threshold = 1.3 기준으로 단어 사전 구축
hawkish_lexicon = final_lexicon[final_lexicon['polarity_score'] > 1.3]
dovish_lexicon = final_lexicon[final_lexicon['polarity_score'] < (1/1.3)]

print(f"hawkish 단어: {len(hawkish_lexicon)}개 발견")
print(f"dovish 단어: {len(dovish_lexicon)}개 발견")

hawkish_set = set(hawkish_lexicon.index)
dovish_set = set(dovish_lexicon.index)

hawkish 단어: 9264개 발견
dovish 단어: 8522개 발견


In [14]:
# 사전 저장
import os

# 1. hawkish와 dovish 라벨 붙이기
hawkish_lexicon = hawkish_lexicon.copy()
hawkish_lexicon['label'] = 'hawkish'

dovish_lexicon = dovish_lexicon.copy()
dovish_lexicon['label'] = 'dovish'

# 2. 두 표를 위아래로 합치기 (concat)
master_lexicon = pd.concat([hawkish_lexicon, dovish_lexicon])

# 3. 저장 경로 설정
save_path = r"C:\KDT\bok_pjt\team2_forecast_pjt\db\lexicon"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 4. 하나의 파일로 저장!
master_lexicon.to_csv(os.path.join(save_path, 'total_lexicon.csv'), encoding='utf-8-sig', index=True)

print(f"✨ 총 {len(master_lexicon)}개의 단어가 담긴 마스터 사전이 저장되었습니다!")

✨ 총 17786개의 단어가 담긴 마스터 사전이 저장되었습니다!
